<a href="https://colab.research.google.com/github/sauravakolia/Fastai2/blob/master/02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!git clone https://github.com/sauravakolia/Fastai2.git

Cloning into 'Fastai2'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 17), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [7]:
import sys
sys.path.append('/content/Fastai2/')

In [8]:
# import operator

# def test(a,b,cmp,cname=None):
#     if cname is None: cname=cmp.__name__
#     assert cmp(a,b),f"{cname}:\n{a}\n{b}"

# def test_eq(a,b): test(a,b,operator.eq,'==')


In [9]:
# def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
# def test_near(a,b): test(a,b,near)

In [10]:
# from pathlib import Path
# from IPython.core.debugger import set_trace
# from fastai import datasets
# import pickle, gzip, math, torch, matplotlib as mpl
# import matplotlib.pyplot as plt
# from torch import tensor

# MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [11]:
# path = datasets.download_data(MNIST_URL, ext='.gz');
# path

In [12]:
#export
from exp.nb_01 import *
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s


In [13]:
x_train,y_train,x_valid,y_valid = get_data()

In [14]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [15]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [16]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [17]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [18]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [19]:

n,m = x_train.shape
c = y_train.max()+1
n,m,c


(50000, 784, tensor(10))

In [20]:

nh = 50



Basic architecture

In [21]:
w1=torch.randn(m,nh)/math.sqrt(m)
b1=torch.zeros(nh)
w2=torch.randn(m,1)/math.sqrt(m)
b2=torch.zeros(1)

In [22]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [23]:
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [24]:
def lin(x,w,b):return x@w+b

In [25]:
t=lin(x_valid,w1,b1)

In [26]:
t.mean(),t.std()

(tensor(0.1287), tensor(1.0051))

In [27]:
def relu(x): return x.clamp_min(0.0)

In [28]:
t = relu(lin(x_valid, w1, b1))

In [29]:
t.mean(),t.std()

(tensor(0.4677), tensor(0.6274))

In [30]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)


In [31]:
w1.mean(),w1.std()

(tensor(3.8254e-05), tensor(0.0506))

In [32]:
t = relu(lin(x_valid, w1, b1))

In [33]:
t.mean(),t.std()

(tensor(0.5299), tensor(0.7740))

In [34]:
#export
from torch.nn import init


In [35]:
w1=torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [36]:
nn.init.kaiming_normal_??

Object `nn.init.kaiming_normal_` not found.


In [37]:
w1.mean(),w1.std()

(tensor(-0.0005), tensor(0.0506))

In [38]:
t.mean(),t.std()

(tensor(0.5804), tensor(0.8080))

In [39]:
def relu(x): return x.clamp_min(0.) - 0.5

In [40]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t = relu(lin(x_valid, w1, b1))
w2 = torch.randn(nh,1)*math.sqrt(2./m )

In [41]:
t.mean(),t.std()

(tensor(0.0792), tensor(0.8123))

In [42]:
def model(x):
  l1=lin(x,w1,b1)
  t1=relu(l1)
  l2=lin(t1,w2,b2)
  return l2


In [43]:
%timeit -n 10 _=model(x_valid)


10 loops, best of 3: 19.4 ms per loop



Loss function: MSE


In [44]:
(model(x_valid).shape)

torch.Size([10000, 1])

In [45]:
model(x_valid).squeeze(-1).shape

torch.Size([10000])

In [46]:
#export
def mse(output,target): return (output.squeeze(-1)-target).pow(2).mean()

In [47]:
y_train,y_valid = y_train.float(),y_valid.float()

In [48]:
preds = model(x_train)

In [49]:
mse(preds,y_train)  

tensor(26.3158)


Gradients and backward pass


In [50]:
def mse_grad(inp,targ):
  inp.g=2.*(inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0]


In [51]:
def relu_grad(inp,out):
  inp.g=(inp>0).float()*out.g

In [52]:
def lin_grad(inp,out,w,b):
  inp.g=out.g @w.t()
  w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
  b.g=out.g.sum(0)

In [53]:
def forward_and_backward(inp,targ):
  # forward function
  l1=inp@w1 +b1
  l2=relu(l1)
  out=l2@w2+b2

  
  # loss func
  loss=mse(out,targ)

  #backward
  mse_grad(out,targ)

  lin_grad(l2,out,w2,b2)

  relu_grad(l1,l2)

  lin_grad(inp,l1,w1,b1)


In [54]:

forward_and_backward(x_train,y_train)

In [55]:
w1g = w1.g.clone()
w2g=w2.g.clone()
b1g = b1.g.clone()
b2g=b2.g.clone()
ig  = x_train.g.clone()

In [56]:
xt2 = x_train.clone().requires_grad_(True)
w12=w1.clone().requires_grad_(True)
w22=w2.clone().requires_grad_(True)
b12=b1.clone().requires_grad_(True)
b22=b2.clone().requires_grad_(True)


In [57]:
def forward(inp,targ):
  # forward function
  l1=inp@w12 +b12
  l2=relu(l1)
  out=l2@w22+b22
  return mse(out,targ)

In [58]:
loss=forward(x_train,y_train)

In [59]:
loss.backward()

In [60]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
# test_near(xt2.grad, ig )

Refractor

In [61]:
class Relu():
  def __call__(self,inp):
    self.inp=inp
    self.out = inp.clamp_min(0.)-0.5
    return self.out

  def backward(self):self.inp.g=(self.inp>0).float()*self.out.g


# class Relu():
#     def __call__(self, inp):
#         self.inp = inp
#         self.out = inp.clamp_min(0.)-0.5
#         return self.out
    
#     def backward(self): self.inp.g = (self.inp>0).float() * self.out.g


In [62]:
class Linear():
  def __init__(self,w,b):
    self.w=w
    self.b=b

  def __call__(self,inp):
    self.inp=inp
    self.out=self.inp@self.w + self.b
    return self.out

  def backward(self):
    self.inp.g=self.out.g @ self.w.t()
    self.w.g=(self.inp.unsqueeze(-1) * self.out.unsqueeze(1)).sum(0)
    self.b.g=self.out.g.sum(0)

# class Lin():
#     def __init__(self, w, b): self.w,self.b = w,b
        
#     def __call__(self, inp):
#         self.inp = inp
#         self.out = inp@self.w + self.b
#         return self.out
    
#     def backward(self):
#         self.inp.g = self.out.g @ self.w.t()
#         # Creating a giant outer product, just to sum it, is inefficient!
#         self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
#         self.b.g = self.out.g.sum(0)



In [63]:
class Mse():
  def __call__(self, inp,targ):
    self.inp=inp
    self.targ=targ
    self.out=(inp.squeeze()-targ).pow(2).mean()
    return self.out

  def backward(self):
    self.inp.g=2.*(self.inp.squeeze()-self.targ).unsqueeze(-1)/self.inp.shape[0]


# class Mse():
#     def __call__(self, inp, targ):
#         self.inp = inp
#         self.targ = targ
#         self.out = (inp.squeeze() - targ).pow(2).mean()
#         return self.out
    
#     def backward(self):
#         self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [64]:
class Model():
  def __init__(self,w1,w2,b1,b2):
    self.layers=[Linear(w1,b1), Relu(), Linear(w2,b2)] 
    self.loss=Mse()   

  def __call__(self,inp,targ):
    for l in self.layers: inp=l(inp)
    self.loss(inp,targ)
 
  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers):
      l.backward()


# class Model():
#     def __init__(self, w1, b1, w2, b2):
#         self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
#         self.loss = Mse()
        
#     def __call__(self, x, targ):
#         for l in self.layers: x = l(x)
#         return self.loss(x, targ)
    
#     def backward(self):
#         self.loss.backward()
#         for l in reversed(self.layers): l.backward()

In [65]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1,w2,b1, b2)
x_train.shape

torch.Size([50000, 784])

In [66]:
%time loss = model(x_train,y_train)

CPU times: user 107 ms, sys: 77 µs, total: 107 ms
Wall time: 107 ms


In [67]:
%time model.backward()

CPU times: user 3.38 s, sys: 3.61 s, total: 6.99 s
Wall time: 7.03 s


Module.forward()

In [68]:
# class Module():
#   # def __call__(self, *args):
#   #       self.args = args
#   #       self.out = self.forward(*args)
#   #       return self.out
#   def __call__(self,*args):
#     self.args=args
#     self.out=self.forward
#     return self.out

#   def forward(self):raise Exception('Not Implemented')

  
#   def backward(self):self.bwd(self.out,*self.args)

class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [69]:
# class Relu(Module):
#   def forward(self,inp): return self.inp.clamp_min(0.)-0.5

#   def bwd(self,out,inp):self.inp.g=(inp>0).float()*out.g

class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [70]:
# class Linear(Module):
#   def __init__(self,w,b):
#     self.w=w
#     self.b=b

#   def forward(self,inp):
#     return inp@self.w + self.b

#   def bwd(self,inp,out):
#     inp.g=out.g @ self.w.t()
#     self.w.g=torch.einsum("bi,bj->ij", inp, out.g)
#     self.b.g=self.out.g.sum(0)

class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [71]:
# class Mse(Module):
#   def forward(self, inp,targ):
#     (self.inp.squeeze()-targ).pow(2).mean()
#     return (self.inp.squeeze()-targ).pow(2).mean()

#   def bwd(self, out, inp, targ):
#     self.inp.g=2.*(self.inp.squeeze()-self.targ).unsqueeze(-1)/self.inp.shape[0]

class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [72]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [73]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [74]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 128 ms


In [75]:
%time model.backward()

CPU times: user 191 ms, sys: 92.1 ms, total: 283 ms
Wall time: 288 ms


In [76]:
#export
from torch import nn

In [77]:
class Model(nn.Module):
  def __init__(self,n_in,nh,n_out):
    super().__init__()
    self.layers=[nn.Linear(n_in,nh),nn.ReLU(),nn.Linear(nh,n_out)]
    self.loss=mse

  def __call__(self,x,targ):
    for l in self.layers:x=l(x)
    return self.loss(x.squeeze(),targ)  

In [78]:
model=Model(m,nh,1)

In [79]:
%time loss=model(x_train,y_train)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 111 ms


In [80]:
%time loss.backward()

CPU times: user 78.2 ms, sys: 0 ns, total: 78.2 ms
Wall time: 79.4 ms


In [82]:
%cd "/content/Fastai2/"

/content/Fastai2


In [81]:
!./notebook2script.py 02_fully_connected.ipynb

/bin/bash: ./notebook2script.py: No such file or directory
